<a href="https://colab.research.google.com/github/tedheo10/STATS-504/blob/main/slides_blank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
library(tidyverse)
options(repr.plot.width = 10 , repr.plot.height = 6, repr.plot.res = 150,
        jupyter.plot_mimetypes = "image/png")
theme_set(theme_classic())

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


# STATS 504
## Week 2
![babies](https://images.ctfassets.net/cnu0m8re1exe/PYLeBWr2QDF5yjyZwB5Iq/7509e49d1d621860c1d4c484243c23db/shutterstock_189429203.jpg)

The US Social Security Administration keeps records of every child born in the United States each year. These records are available in aggregate form in a [public database](https://www.ssa.gov/oact/babynames/).

We are interested in understanding more about baby names

In [ ]:
#install.packages("babynames")
library(babynames)
babynames %>% filter(year < 1890, name == "Mary", sex == "F") %>% summarize(s = sum(n))


The downloaded binary packages are in
	/var/folders/ng/qcb7p5f14nl6kwpxnh6rz73h0000gn/T//RtmpVqSwii/downloaded_packages


s
<int>
91668


In [ ]:
babynames <- read_csv("babynames.csv.gz")

Rows: 1924665 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): sex, name
dbl (3): year, n, prop

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


<img src="https://beam-images.warnermediacdn.com/BEAM_LWM_DELIVERABLES/52dae4c7-2ab1-4bb9-ab1c-8100fd54e2f9/525e19cadd275c68e933773cc3bdad91a12fe10e.jpg?submitted=1684658977805&host=wbd-images.prod-vod.h264.io&partner=beamcom">

## Eternal names
The most common name each year is:

We see that Mary and John were extremely popular names in the late 19th century.

## &#129300; Quiz

Call a name *eternal* if it is among the most 30 common names for its gender in *every* year of data.

John is eternal, but Mary is not. Including John there are four eternal boy names, and one eternal girl name.

What is the eternal girl's name?

<ol style="list-style-type: upper-alpha;">
    <li>Anna</li>
    <li>Elizabeth</li>
    <li>Khloe</li>
    <li>Spike</li>
    <li>Margaret</li>
</ol>

## Naming trends driven by migration
What fraction of male names ended in "o", or female names in "a", over time?

In [ ]:
stringr::str_sub('km', -1)

[1] "m"

## First letter
Does the first letter of a name show evidence of trends in popularity over time?

I find this plot hard to interpret (too busy). Perhaps a table would work better. Let's look at the most common first letter by decade:

## Naming diversity
Has the variety of names increased over time? How can we measure this?

- Number of unique names/year
- Entropy
- Inequality measure (Gini coefficient)

## Entropy
The entropy of a discrete probability distribution is:

$$H = -\sum_i p_i \log p_i$$

## Gini's coefficient

Gini's coefficient is another measure of inequality in a distribution. It is defined as:

$$G = \frac{\sum_{i,j} |x_i - x_j|}{2 \bar{x} n^2}$$



Assuming $x_i\ge 0$:

- $G=0$: perfect equality.
- $G=1$: "perfect inequality": one person has everything, everyone else has nothing.
- $G \ge 0$: obvious.
- $G \le 1$: obvious?

In [ ]:
gini <- function(xs) {
    n <- length(xs)
    D <- dist(xs, diag=T, upper=T) %>% as.matrix
    mean(D) / 2 / mean(xs)
}

gini(c(1,1,123))

[1] 0.6506667

In [ ]:
install.packages('dineq')
library(dineq)
gini.wtd(c(10000000000,1,1,1,1,1))

also installing the dependencies ‘checkmate’, ‘htmlTable’, ‘viridis’, ‘Hmisc’





The downloaded binary packages are in
	/var/folders/ng/qcb7p5f14nl6kwpxnh6rz73h0000gn/T//RtmpVqSwii/downloaded_packages


[1] 0.8333333

The Gini index for both male and female names declines over time, indicating that name choices have become more diverse.

Historically, baby names were concentrated around a small set of traditional names (high Gini), but in recent decades, naming has become less concentrated, with a wider variety of names being used (lower Gini).

Female names generally show a lower Gini index than male names, reflecting greater diversity and faster changing trends in naming preferences for girls.


## Age vs. name
Can you accurately predict someone's age based on their name and sex at birth? How should we approach this question?

## Modeling
When predicting someone's age based on name, we have to take into account two factors:

- When were the most babies born with that name?
- What is the likelihood that a person born in that year is still alive today?

This naturally leads to an application of Bayes rule.

## Bayesian model




\begin{gather}
\mathbb{P}(\text{age}=a\mid\text{name})=\mathbb{P}(\text{birth year}=2024-a\mid\text{name})
\\=\mathbb{P}(\text{name}\mid\text{birth year}=2024-a)\frac{\mathbb{P}(\text{birth year}=2024-a)}{\mathbb{P}(\text{name})} \\
\propto  \mathbb{P}(\text{name}\mid\text{birth year}=2024-a)\mathbb{P}(\text{birth year}=2024-a)
\end{gather}

What are the quantities we would need to estimate?

- $\mathbb{P}(\text{age}=a\mid\text{name})=\mathbb{P}(\text{birth year}=2024-a\mid\text{name})$:  ✅
- $\mathbb{P}(\text{birth year}=2024-a)$: the age distribution of the current population.

## Age distribution of the current population
- The US Census has this information [readily available](https://www.census.gov/popclock/data_tables.php?component=pyramid).
- (I downloaded the table by hand since their website makes it hard to automate.)

In [ ]:
# colab
# us_pop <- read_csv("https://raw.githubusercontent.com/terhorst/stats504/main/lectures/week2/United%20States%20Population%20by%20Age%20and%20Sex.csv")

read_csv('United States Population by Age and Sex.csv', skip=1) %>%
    mutate(across(1:3, \(x) str_replace_all(x, "[^\\d.]", "") %>% as.numeric)) %>%
    slice(1:101) -> ages

ages

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details, e.g.:
  dat <- vroom(...)
  problems(dat)”
Rows: 307 Columns: 3
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (3): Age, Male % of Population, Female % of Population

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Age,Male % of Population,Female % of Population
<dbl>,<dbl>,<dbl>
0,0.56,0.54
1,0.55,0.53
2,0.57,0.54
3,0.57,0.55
4,0.59,0.56
5,0.60,0.57
6,0.61,0.59
7,0.62,0.59
8,0.62,0.59


## Putting it together